## Import

In [48]:
# !pip3 install albumentations ;!pip3 install opencv-python ; !pip3 install tqdm

In [73]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# torch.multiprocessing import
from torch import multiprocessing

import warnings
warnings.filterwarnings(action='ignore') 

In [74]:
# 경로지정
import os
os.chdir('../DATA')
os.getcwd()

'/Users/admin/Documents/GitHub/Dacon_papering_classification/DATA'

In [75]:
# seeds
random_seed = 42
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [76]:
# device 설정
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
# mps 확인
torch.backends.mps.is_available()


True

In [77]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
import os

# 데이터셋 디렉토리 경로
dataset_dir = "train"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    item_path = os.path.join(dataset_dir, item)
    for file in os.listdir(item_path):
        all_img_list.append(os.path.join(item_path, file))
        folder_list.append(item)
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[1])
from collections import Counter
Counter(df['label'])


Counter({'녹오염': 14,
         '걸레받이수정': 307,
         '꼬임': 210,
         '석고수정': 57,
         '오타공': 142,
         '울음': 22,
         '이음부불량': 17,
         '몰딩수정': 130,
         '면불량': 99,
         '창틀,문틀수정': 27,
         '피스': 51,
         '곰팡이': 145,
         '반점': 3,
         '들뜸': 54,
         '오염': 595,
         '가구수정': 12,
         '터짐': 162,
         '훼손': 1405,
         '틈새과다': 5})

In [146]:
# hyperparameter
CFG = {
    'IMG_SIZE':64,
    'EPOCHS':20,
    'LEARNING_RATE':0.01,
    'BATCH_SIZE':1,
    'SEED':42
}

In [79]:
os.getcwd()

'/Users/admin/Documents/GitHub/Dacon_papering_classification/DATA'

In [80]:
# 낮은 순
folder_name = Counter(df['label']).most_common()[::-1][0][0]
folder_name = f'train_aug/{folder_name}'
# make folder if not exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f'{folder_name} 폴더 생성 완료')

df_aug = df[df['label']==Counter(df['label']).most_common()[::-1][0][0]].reset_index(drop=True)
df_aug = df_aug.reset_index(drop=True)
df_aug['label']= 1
# df_aug = df_aug[0:1]
df_aug

,img_path,label
0,train/반점/2.png,1
1,train/반점/1.png,1
2,train/반점/0.png,1


In [81]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [82]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
 
# test_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
#                             ToTensorV2()
#                             ])
train_dataset = CustomDataset(df_aug['img_path'].values, df_aug['label'].values, train_transform)
data_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
# val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [83]:
next(iter(data_loader))[0].shape

torch.Size([1, 3, 64, 64])

In [84]:
next(iter(train_dataset))[1]

1

In [154]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_dim=100, output_channels=3):
        super(Generator, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 128 * 8 * 8)
        self.conv1 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(64, output_channels, kernel_size=4, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        
        out = self.fc2(out)
        out = self.relu(out)
        
        out = out.view(out.shape[0], 128, 8, 8)
        
        out = self.conv1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.tanh(out)
        
        return out

# Check the size of each layer
input_dim = 100
output_channels = 3
batch_size = 1
z = torch.randn(batch_size, input_dim)

generator = Generator(input_dim, output_channels)
print(generator)

x = z
x = generator.fc1(x)
print(x.shape) # expected (batch_size, 1024)

x = generator.fc2(x)
print(x.shape) # expected (batch_size, 128*8*8)

x = x.view(x.shape[0], 128, 8, 8)
print(x.shape) # expected (batch_size, 128, 8, 8)

x = generator.conv1(x)
print(x.shape) # expected (batch_size, 64, 16, 16)

x = generator.conv2(x)
print(x.shape) # expected (batch_size, output_channels, 32, 32)


Generator(
  (fc1): Linear(in_features=100, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=8192, bias=True)
  (conv1): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (relu): ReLU()
  (tanh): Tanh()
)
torch.Size([1, 1024])
torch.Size([1, 8192])
torch.Size([1, 128, 8, 8])
torch.Size([1, 64, 16, 16])
torch.Size([1, 3, 32, 32])


In [162]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_dim=100, output_channels=3):
        super(Generator, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 128 * 8 * 8)
        self.conv1 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(64, output_channels, kernel_size=4, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        
        out = self.fc2(out)
        out = self.relu(out)
        
        out = out.view(out.shape[0], 128, 8, 8)
        
        out = self.conv1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.tanh(out)
        
        return out

# Check the size of each layer
input_dim = 100
output_channels = 3
batch_size = 1
z = torch.randn(batch_size, input_dim)

generator = Generator(input_dim, output_channels)
print(generator)

x = z
x = generator.fc1(x)
print(x.shape) # expected (batch_size, 1024)

x = generator.fc2(x)
print(x.shape) # expected (batch_size, 128*8*8)

x = x.view(x.shape[0], 128, 8, 8)

print(x.shape) # expected (batch_size, 128, 8, 8)

x = generator.conv1(x)
print(x.shape) # expected (batch_size, 64, 16, 16)

x = generator.conv2(x)
print(x.shape) # expected (batch_size, output_channels, 32, 32)


Generator(
  (fc1): Linear(in_features=100, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=8192, bias=True)
  (conv1): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (relu): ReLU()
  (tanh): Tanh()
)
torch.Size([1, 1024])
torch.Size([1, 8192])
torch.Size([1, 128, 8, 8])
torch.Size([1, 64, 16, 16])
torch.Size([1, 3, 32, 32])


In [103]:


# Initialize generator and discriminator networks
generator = Generator()
discriminator = Discriminator()

# Define loss function and optimizers
criterion = nn.BCELoss()
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader, start=0):
        # Train the discriminator with real images
        real_labels = torch.ones(images.size(0), 1)
        fake_labels = torch.zeros(images.size(0), 1)
        
        discriminator_real_outputs = discriminator(images)
        
        
        # Train the discriminator with fake images generated by the generator
        z = torch.randn(images.size(0), latent_size)
        fake_images = generator(z)
        
        discriminator_fake_outputs = discriminator(fake_images.detach())
        discriminator_fake_loss = criterion(discriminator_fake_outputs, discriminator_real_outputs)
        discriminator_real_loss = criterion(fake_labels, real_labels)
        discriminator_loss = discriminator_real_loss + discriminator_fake_loss
        discriminator.zero_grad()
        discriminator_loss.backward()
        discriminator_optimizer.step()
        
        # Train the generator
        z = torch.randn(images.size(0), latent_size)
        fake_images = generator(z)
        
        generator_outputs = discriminator(fake_images)
        generator_loss = criterion(generator_outputs, real_labels[:fake_images.size(0)])

        generator.zero_grad()
        generator_loss.backward()
        generator_optimizer.step()
        
        if (epoch+1) % 10 == 0 and i == 0:
            with torch.no_grad():
                z = torch.randn(25, latent_size)
                fake_images = generator(z)
                fake_images = fake_images.view(fake_images.size(0), 3, 64, 64)  # apply IMG_SIZE from CFG
                save_image(fake_images, f"images/{epoch+1}.png", nrow=5)

    # Print progress
    print(f"Epoch [{epoch+1}/{num_epochs}], Generator Loss: {generator_loss.item():.4f}, Discriminator Loss: {discriminator_loss.item():.4f}")

# Save final generator model
torch.save(generator.state_dict(), 'generator.pth')



ValueError: Using a target size (torch.Size([1, 3, 64, 1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.

In [1]:
import gc
gc.collect()

50

In [161]:
class Generator(nn.Module):
    def __init__(self, input_dim=100, output_channels=3):
        super(Generator, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 128 * 8 * 8)
        self.conv1 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose2d(64, output_channels, kernel_size=4, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        
        out = self.fc2(out)
        out = self.relu(out)
        
        out = out.view(out.shape[0], 128, 8, 8)
        
        out = self.conv1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.tanh(out)
        
        return out

# Check the size of each layer
input_dim = 100
output_channels = 3
batch_size = 1
z = torch.randn(batch_size, input_dim)

generator = Generator(input_dim, output_channels)
print(generator)

for epoch in range(3):
    for i, (images, labels) in enumerate(data_loader, start=0):
        # Train the discriminator with real images
        real_labels = torch.ones(images.size(0), 1)
        fake_labels = torch.zeros(images.size(0), 1)
        # print(images.shape)
        discriminator_real_outputs = discriminator(images)
        print(f'discriminator_real_outputs shape is {discriminator_real_outputs.shape}')
       
        # Train the discriminator with fake images generated by the generator
        z = torch.randn(images.size(0), 3, 1, 1)
        z = z.expand(images.size(0), 3, 64, 64)

        print(f'z shape is {z.shape}')
     
        fake_images = generator(z)
        discriminator_fake_outputs = discriminator(fake_images.detach())
       
        # # Compute the discriminator loss
        # discriminator_real_loss = criterion(discriminator_real_outputs, real_labels)
        # discriminator_fake_loss = criterion(discriminator_fake_outputs, fake_labels)
        # discriminator_loss = discriminator_real_loss + discriminator_fake_loss


Generator(
  (fc1): Linear(in_features=100, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=8192, bias=True)
  (conv1): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (relu): ReLU()
  (tanh): Tanh()
)
discriminator_real_outputs shape is torch.Size([1, 1])
z shape is torch.Size([1, 3, 64, 64])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (192x64 and 100x1024)

In [138]:
next(iter(data_loader))[0].shape

torch.Size([1, 3, 64, 64])

## Hyperparameter Setting

In [7]:
# hyperparameter
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':2022
}

## Fixed RandomSeed

In [8]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [10]:
os.getcwd()

'/Users/admin/Documents/GitHub/Dacon_papering_classification/DATA'

In [11]:
import os

# 데이터셋 디렉토리 경로
dataset_dir = "train"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    item_path = os.path.join(dataset_dir, item)
    for file in os.listdir(item_path):
        all_img_list.append(os.path.join(item_path, file))
        folder_list.append(item)

In [12]:
all_img_list[:5]

['train/18/348.png',
 'train/18/1186.png',
 'train/18/412.png',
 'train/18/374.png',
 'train/18/360.png']

In [13]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[1])
df

,img_path,label
0,train/18/348.png,18
1,train/18/1186.png,18
2,train/18/412.png,18
3,train/18/374.png,18
4,train/18/360.png,18
...,...,...
3452,train/14/6.png,14
3453,train/14/2.png,14
3454,train/14/3.png,14
3455,train/14/1.png,14


In [14]:
from sklearn.model_selection import train_test_split

train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])


## Label-Encoding

In [116]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])
train

,img_path,label
3412,train/damage/1014.png,3
1147,train/wrinkle/30.png,18
3327,train/damage/781.png,3
3064,train/damage/1117.png,3
1816,train/contamination/494.png,0
...,...,...
126,train/typo/137.png,15
2316,train/damage/417.png,3
1146,train/wrinkle/18.png,18
80,train/typo/73.png,15


## CustomDataset

In [117]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
# transformer model
train_transforms = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(),
    ToTensorV2()
])
de

In [118]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [119]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [120]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b4(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [121]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [122]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [18]:
#!/bin/bash
# os.chdir('train')
# 변경 전 폴더명 리스트
old_names = ["녹오염", "걸레받이수정", "꼬임", "석고수정", "오타공", "울음", "이음부불량",
             "몰딩수정", "면불량", "창틀,문틀수정", "피스", "곰팡이", "반점", "들뜸", "오염",
             "가구수정", "터짐", "훼손", "틈새과다"]

# 변경 후 폴더명 리스트
new_names = ["rust", "mop_cleaning", "twist", "gypsum_repair", "typo", "cry", "splice_defect",
             "molding_repair", "wrinkle", "window_frame_repair", "piece", "mold", "stain", 
             "uneven", "contamination", "furniture_repair", "crack", "damage", "excessive_gap"]

ch_names = os.listdir()
# 현재 폴더 경로
current_dir = os.getcwd()

# 폴더명 변경
for old_name, new_name in zip(ch_names, old_names):
    old_path = old_name
    new_path = new_name
    os.rename(old_path, new_path)



## Run!!

In [124]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /Users/admin/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth
100%|██████████| 74.5M/74.5M [00:06<00:00, 12.5MB/s]


  0%|          | 0/87 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 7.63 GB, other allocations: 1.43 GB, max allowed: 9.07 GB). Tried to allocate 9.76 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

## Inference

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
# result folder 생성
from datetime import datetime, timezone, timedelta

kst = timezone(timedelta(hours=9))
train_serial =  datetime.now(kst).strftime('%Y%m%d_%H%M%S')

Record_path = os.path.join('../result', train_serial)

os.makedirs(Record_path, exist_ok=True)




In [ ]:
# 변경 전 폴더명 리스트
old_names = ["녹오염","걸레받이수정","꼬임","석고수정","오타공", "울음", "이음부불량",
             "몰딩수정", "면불량", "창틀,문틀수정", "피스", "곰팡이", "반점", "들뜸", "오염",
             "가구수정", "터짐", "훼손", "틈새과다"]

# 변경 후 폴더명 리스트
new_names = ["rust", "mop_cleaning", "twist", "gypsum_repair", "typo", "cry", "splice_defect",
             "molding_repair", "wrinkle", "window_frame_repair", "piece", "mold", "stain", 
             "uneven", "contamination", "furniture_repair", "crack", "damage", "excessive_gap"]

# Dictionary to map old column names to new column names
name_map = dict(zip(new_names, old_names))
name_map


# Rename columns using the rename() method
submit_test = submit.replace(name_map)
submit_test

In [ ]:
submit_test.to_csv(os.path.join(Record_path,'baseline_submit.csv'), index=False)

In [ ]:
submit_test

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,꼬임
788,TEST_788,오염
789,TEST_789,오염
790,TEST_790,오염
